### Tokenization

In [70]:
import random
from nltk.corpus import gutenberg
from nltk.tokenize import word_tokenize
from collections import Counter, defaultdict
from nltk.util import ngrams
import math

import nltk

nltk.download('gutenberg')
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package gutenberg to
[nltk_data]     /Users/rattanak/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package punkt to /Users/rattanak/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/rattanak/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [71]:
# Load the Gutenberg Corpus (or replace with your own text corpus)
# For demonstration, combining a few texts from the Gutenberg corpus
corpus = (
    # gutenberg.raw('crawler/data/wikipedia.txt')
    gutenberg.raw('austen-emma.txt') +
    gutenberg.raw('austen-sense.txt') +
    gutenberg.raw('bible-kjv.txt')
)

In [72]:
# Step 2: Split the Corpus into Subsets
# Shuffle the sentences to ensure random sampling
corpus = corpus.replace('--', '').replace('``', '').replace("''", '').replace('`', '')

sentences = corpus.split('\n')
random.shuffle(sentences)

In [73]:
# Calculate subset sizes
total_sentences = len(sentences)
train_size = int(0.7 * total_sentences)
val_size = int(0.1 * total_sentences)
test_size = total_sentences - train_size - val_size

# Create subsets
train_sentences = sentences[:train_size]
val_sentences = sentences[train_size:train_size + val_size]
test_sentences = sentences[train_size + val_size:]

# Combine sentences back into text for tokenization
train_text = ' '.join(train_sentences)
val_text = ' '.join(val_sentences)
test_text = ' '.join(test_sentences)

In [74]:
print(train_sentences[:8]),
print("Length: ", total_sentences)

['', '', 'of them, in earnest conversation with a very fashionable', '', 'without blemish: 28:20 And their meat offering shall be of flour', 'cometh to God must believe that he is, and that he is a rewarder of', 'loose the sackcloth from off thy loins, and put off thy shoe from thy', '']
Length:  131425


In [75]:
# Tokenize the Text
train_tokens = word_tokenize(train_text.lower())
val_tokens = word_tokenize(val_text.lower())
test_tokens = word_tokenize(test_text.lower())
# train_tokens = word_tokenize(train_text.lower())

In [76]:
# Limit Vocabulary Size and Replace Rare Tokens with <UNK>
# Define maximum vocabulary size
MAX_VOCAB_SIZE = 5000

# Count word frequencies in the training set
word_counts = Counter(train_tokens)

# Select the most common words for the vocabulary
vocab = {word for word, _ in word_counts.most_common(MAX_VOCAB_SIZE)}

# Replace rare tokens with <UNK>
def replace_with_unk(tokens, vocab):
    return [token if token in vocab else "<UNK>" for token in tokens]

train_tokens_limited = replace_with_unk(train_tokens, vocab)
val_tokens_limited = replace_with_unk(val_tokens, vocab)
test_tokens_limited = replace_with_unk(test_tokens, vocab)

# Results
print(f"Original Training Tokens: {len(train_tokens)}")
print(f"Training Tokens After Limiting Vocabulary: {len(train_tokens_limited)}")
print(f"Example Training Tokens: {train_tokens_limited[:50]}")

Original Training Tokens: 891920
Training Tokens After Limiting Vocabulary: 891920
Example Training Tokens: ['of', 'them', ',', 'in', 'earnest', 'conversation', 'with', 'a', 'very', '<UNK>', 'without', 'blemish', ':', '28:20', 'and', 'their', 'meat', 'offering', 'shall', 'be', 'of', 'flour', 'cometh', 'to', 'god', 'must', 'believe', 'that', 'he', 'is', ',', 'and', 'that', 'he', 'is', 'a', '<UNK>', 'of', 'loose', 'the', 'sackcloth', 'from', 'off', 'thy', 'loins', ',', 'and', 'put', 'off', 'thy']


In [77]:
# Add <s> (start) and </s> (end) tokens for sentence boundaries
"""
A 4-gram model generates n-grams of length 4. For the first token in a sentence, there are no preceding words. 
To address this, multiple <s> tokens are added to represent the missing preceding words, ensuring that every word in the sentence can appear as part of a valid 4-gram.
"""
train_tokens_limited = ["<s>", "<s>", "<s>"] + train_tokens_limited + ["</s>"]

In [78]:
print(train_tokens_limited[:100])

['<s>', '<s>', '<s>', 'of', 'them', ',', 'in', 'earnest', 'conversation', 'with', 'a', 'very', '<UNK>', 'without', 'blemish', ':', '28:20', 'and', 'their', 'meat', 'offering', 'shall', 'be', 'of', 'flour', 'cometh', 'to', 'god', 'must', 'believe', 'that', 'he', 'is', ',', 'and', 'that', 'he', 'is', 'a', '<UNK>', 'of', 'loose', 'the', 'sackcloth', 'from', 'off', 'thy', 'loins', ',', 'and', 'put', 'off', 'thy', '<UNK>', 'from', 'thy', 'fountain', 'of', 'living', 'waters', ',', 'and', 'hewed', 'them', 'out', '<UNK>', ',', 'broken', 'man', 'that', 'hath', 'left', 'house', ',', 'or', 'brethren', ',', 'or', 'sisters', ',', 'or', 'father', ',', 'or', 'candlestick', ',', 'and', 'also', 'for', 'the', 'lamps', 'thereof', ',', 'according', 'to', 'the', 'use', 'of', '<UNK>', 'who']


In [79]:
# Count n-grams
unigram_counts = Counter(ngrams(train_tokens_limited, 1))
bigram_counts = Counter(ngrams(train_tokens_limited, 2))
trigram_counts = Counter(ngrams(train_tokens_limited, 3))
fourgram_counts = Counter(ngrams(train_tokens_limited, 4))

# Total unigrams (needed for probabilities)
total_unigrams = sum(unigram_counts.values())

# Vocabulary size (for smoothing)
vocab_size = len(set(train_tokens_limited))

In [80]:
# gram = ('<s>', '<s>', '<s>', 'your')
# print(len(gram), fourgram_counts[('<UNK>', ',', 'and', '<UNK>')])
# fourgram_counts[gram],
# # trigram_counts[gram[:3]]
# fourgram_counts
set(train_tokens_limited)

{'communed',
 'satan',
 "''",
 'death',
 '12:11',
 'governors',
 '12:1',
 'ahaz',
 'praised',
 'bates',
 'a-year',
 'borders',
 'own',
 'galilee',
 'fulfil',
 'poverty',
 'levi',
 'seemeth',
 'distance',
 'everything',
 '17:8',
 'set',
 'language',
 'difference',
 'bowels',
 'owed',
 'confess',
 'agitated',
 'seventy',
 'entering',
 'cities',
 'wickedly',
 '16:20',
 'refuse',
 'shocked',
 '5:27',
 'perceiving',
 'sing',
 'sinner',
 'choose',
 'inner',
 'tomorrow',
 'jedaiah',
 '19:11',
 'treated',
 'established',
 'singing',
 'encouragement',
 'fish',
 '25:8',
 'punished',
 'fellows',
 'persuaded',
 'rend',
 'concern',
 'direct',
 '21:17',
 'miserable',
 'contend',
 '13:32',
 'knowing',
 'publicans',
 'siege',
 '12:4',
 '3:13',
 'on',
 'avenged',
 'sojourn',
 'heavy',
 'among',
 '16:10',
 'nineveh',
 'lamentation',
 'gave',
 'conscience',
 '18:21',
 'true',
 'princes',
 'influence',
 'attentive',
 'fowls',
 'sepulchre',
 'snare',
 '18:14',
 '8:5',
 'useful',
 '16:25',
 'shot',
 'lusts'

In [81]:
# Define the Backoff Function
def backoff_probability(ngram):
    """
    Compute the probability of an n-gram using backoff (unsmoothed).
    Falls back to lower n-grams if the higher-order n-gram is missing.
    """
    # print("ngrams: ", ngram)
    # 4-gram case
    if len(ngram) == 4 and fourgram_counts[ngram] > 0:
        return fourgram_counts[ngram] / trigram_counts[ngram[:3]]
    
    # 3-gram case
    elif len(ngram) == 4 and trigram_counts[ngram[1:]] > 0:
        return trigram_counts[ngram[1:]] / bigram_counts[ngram[1:3]]
    
    # 2-gram case
    elif len(ngram) == 4 and bigram_counts[ngram[2:]] > 0:
        return bigram_counts[ngram[2:]] / unigram_counts[(ngram[2],)]
    
    # Unigram case (fallback)
    elif len(ngram) == 4 and unigram_counts[(ngram[3],)] > 0:
        return unigram_counts[(ngram[3],)] / total_unigrams
    
    # If no probabilities available, return 0
    return 0

In [82]:
# Test the Backoff Probability Function
def test_backoff(sentences):
    """
    Compute probabilities for given sentences using the backoff 4-gram model.
    """
    for sentence in sentences:
        # Preprocess sentence
        tokens = word_tokenize(sentence.lower())
        tokens = ["<s>", "<s>", "<s>"] + tokens + ["</s>"]
        
        # Create 4-grams
        sentence_ngrams = list(ngrams(tokens, 4))
        
        # Compute probabilities
        print(f"Sentence: {sentence}")
        for ngram in sentence_ngrams:
            prob = backoff_probability(ngram)
            print(f"  {ngram} -> {prob}")
        print()

# Example Sentences for Testing
test_sentences = [
    "Your example sentence goes here.",
    "Another test sentence for the backoff model."
]

# Run the test
test_backoff(test_sentences)

Sentence: Your example sentence goes here.
  ('<s>', '<s>', '<s>', 'your') -> 0.0019721411241316526
  ('<s>', '<s>', 'your', 'example') -> 1.1211717590287962e-05
  ('<s>', 'your', 'example', 'sentence') -> 1.6817576385431942e-05
  ('your', 'example', 'sentence', 'goes') -> 0
  ('example', 'sentence', 'goes', 'here') -> 0.00030047403141971737
  ('sentence', 'goes', 'here', '.') -> 0.13059701492537312
  ('goes', 'here', '.', '</s>') -> 1.1211717590287963e-06

Sentence: Another test sentence for the backoff model.
  ('<s>', '<s>', '<s>', 'another') -> 0.0005000426045268431
  ('<s>', '<s>', 'another', 'test') -> 0
  ('<s>', 'another', 'test', 'sentence') -> 1.6817576385431942e-05
  ('another', 'test', 'sentence', 'for') -> 0.00907140070230199
  ('test', 'sentence', 'for', 'the') -> 0.1635150166852058
  ('sentence', 'for', 'the', 'backoff') -> 0
  ('for', 'the', 'backoff', 'model') -> 0
  ('the', 'backoff', 'model', '.') -> 0.02888138451258179
  ('backoff', 'model', '.', '</s>') -> 1.121171

### b). LM2: Interpolation method. The computation of each n-gram term is also smoothed using add-k smoothing technique. You can conduct your own experiments to find the best values for the hyper-parameters $\lambda$'s and $k$ (use the validation set for the experiments).

In [83]:
# Add-k Smoothing Function
def add_k_smoothing(ngram, ngram_counts, lower_order_counts, k, vocab_size):
    """
    Compute add-k smoothed probability for an n-gram.
    """
    ngram_count = ngram_counts[ngram]
    prefix_count = lower_order_counts[ngram[:-1]] if len(ngram) > 1 else sum(unigram_counts.values())
    return (ngram_count + k) / (prefix_count + k * vocab_size)


# Interpolation Function
def interpolated_probability(ngram, lambdas, k):
    """
    Compute the interpolated probability of a word using 4-gram, 3-gram, 2-gram, and unigram models.
    """
    lambda4, lambda3, lambda2, lambda1 = lambdas
    p4 = add_k_smoothing(ngram, fourgram_counts, trigram_counts, k, vocab_size) if len(ngram) == 4 else 0
    p3 = add_k_smoothing(ngram[1:], trigram_counts, bigram_counts, k, vocab_size) if len(ngram) >= 3 else 0
    p2 = add_k_smoothing(ngram[2:], bigram_counts, unigram_counts, k, vocab_size) if len(ngram) >= 2 else 0
    p1 = add_k_smoothing((ngram[-1],), unigram_counts, None, k, vocab_size)
    return lambda4 * p4 + lambda3 * p3 + lambda2 * p2 + lambda1 * p1

# Tune Hyperparameters Using the Validation Set
def tune_hyperparameters(val_tokens, lambdas_list, k_values):
    """
    Experiment with different values of lambdas and k using the validation set.
    """
    best_lambda = None
    best_k = None
    best_log_prob = float('-inf')

    for lambdas in lambdas_list:
        for k in k_values:
            # Compute log-probability on validation set
            log_prob = 0
            for ngram in ngrams(val_tokens, 4):
                prob = interpolated_probability(ngram, lambdas, k)
                log_prob += math.log(prob) if prob > 0 else float('-inf')
            
            # Update best parameters
            if log_prob > best_log_prob:
                best_log_prob = log_prob
                best_lambda = lambdas
                best_k = k

    return best_lambda, best_k, best_log_prob

# Example: Tune parameters
lambda_options = [
    (0.25, 0.25, 0.25, 0.25),  # Equal weights
    (0.4, 0.3, 0.2, 0.1),      # Higher weight for 4-grams
    (0.1, 0.2, 0.3, 0.4),      # Higher weight for unigrams
]
k_values = [0.1, 0.5, 1.0]

best_lambda, best_k, best_log_prob = tune_hyperparameters(val_tokens, lambda_options, k_values)

print(f"Best Lambda: {best_lambda}")
print(f"Best k: {best_k}")
print(f"Log-Probability: {best_log_prob}")

Best Lambda: (0.1, 0.2, 0.3, 0.4)
Best k: 0.1
Log-Probability: -690902.7502802992


In [84]:
# Test the Model
test_text = "your test text here"
def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    return ["<s>", "<s>", "<s>"] + tokens + ["</s>"]
test_tokens = preprocess_text(test_text)

# Compute probabilities for test sentences
for ngram in ngrams(test_tokens, 4):
    prob = interpolated_probability(ngram, best_lambda, best_k)
    print(f"N-gram: {ngram}, Probability: {prob}")

N-gram: ('<s>', '<s>', '<s>', 'your'), Probability: 0.0009078306061606472
N-gram: ('<s>', '<s>', 'your', 'test'), Probability: 7.328729239305256e-05
N-gram: ('<s>', 'your', 'test', 'text'), Probability: 0.00011997286490295254
N-gram: ('your', 'test', 'text', 'here'), Probability: 0.00024009509823971848
N-gram: ('test', 'text', 'here', '</s>'), Probability: 9.950430777354939e-05


### I. Evaluate the two models on the test set using perplexity evaluation metric.

In [85]:
import math

# Perplexity Calculation Function
def calculate_perplexity(test_tokens, model, lambdas=None, k=None):
    """
    Calculate perplexity of a language model on the test set.
    
    Parameters:
    - test_tokens: List of tokens in the test set.
    - model: Function to compute n-gram probabilities.
    - lambdas: Interpolation weights (only used for LM2).
    - k: Smoothing parameter (only used for LM2).
    
    Returns:
    - Perplexity value.
    """
    log_prob_sum = 0
    total_ngrams = 0

    # Generate 4-grams from test tokens
    test_ngrams = list(ngrams(test_tokens, 4))

    for ngram in test_ngrams:
        # Get the probability from the model
        if lambdas and k:
            # LM2 (interpolation with add-k smoothing)
            prob = interpolated_probability(ngram, lambdas, k)
        else:
            # LM1 (backoff without smoothing)
            prob = backoff_probability(ngram)

        # If probability is 0, assign a small probability (to handle unknowns)
        if prob == 0:
            prob = 1e-10  # Small probability for unseen n-grams

        # Accumulate log-probability
        log_prob_sum += math.log(prob)
        total_ngrams += 1

    # Calculate perplexity
    avg_log_prob = log_prob_sum / total_ngrams
    perplexity = math.exp(-avg_log_prob)

    return perplexity

# Preprocess the Test Set
# test_text = "your test text here"
test_tokens = preprocess_text(test_text)  # Add <s>, </s> markers

# Evaluate LM1 (Backoff Model)
lm1_perplexity = calculate_perplexity(test_tokens, model="LM1")
print(f"Perplexity of LM1 (Backoff Model): {lm1_perplexity}")

# Evaluate LM2 (Interpolation with Add-k Smoothing)
lm2_perplexity = calculate_perplexity(test_tokens, model="LM2", lambdas=best_lambda, k=best_k)
print(f"Perplexity of LM2 (Interpolation + Add-k Smoothing): {lm2_perplexity}")


Perplexity of LM1 (Backoff Model): 2725941.8260661173
Perplexity of LM2 (Interpolation + Add-k Smoothing): 5545.384438106823


### II. Create a text generator using each of the two models

In [86]:
import random

# Step 1: Text Generator for LM1 (Backoff Model)
def generate_text_lm1(start_tokens, max_length=20):
    """
    Generate text using LM1 (Backoff Model).
    
    Parameters:
    - start_tokens: List of tokens to start the text generation (e.g., ["<s>", "<s>", "<s>"]).
    - max_length: Maximum length of the generated sequence.
    
    Returns:
    - Generated text as a string.
    """
    generated_tokens = start_tokens[:]
    # print("generated token: ", generated_tokens)
    while len(generated_tokens) < max_length:
        context = tuple(generated_tokens[-3:])  # Use the last 3 tokens as context for 4-grams
        next_token_probs = {}

        # Compute probabilities for the next token
        for token in unigram_counts.keys():
           
            ngram = context + token#(token,)
            # print("ngram: ", ngram)
            prob = backoff_probability(ngram)  # Use LM1's backoff probability function
            if prob > 0:
                next_token_probs[token] = prob

        # print("next token prob: ", next_token_probs)
        # Normalize probabilities
        total_prob = sum(next_token_probs.values())
        normalized_probs = {token: prob / total_prob for token, prob in next_token_probs.items()}
        # print("normalise prob: ", normalized_probs)
        # Sample the next token
        next_token = random.choices(list(normalized_probs.keys()), weights=normalized_probs.values())[0]
        # print("next token: ", next_token)
        # Stop if end token is generated
        if next_token == "</s>":
            break
        
        generated_tokens.extend(list(next_token))
        print("generated token: ", generated_tokens)

    return " ".join(generated_tokens[3:])  # Remove the initial <s> tokens


# Generate Text
start_tokens = ["<s>", "<s>", "<s>"]

# Generate text using LM1
generated_text_lm1 = generate_text_lm1(start_tokens)
print("Generated Text (LM1 - Backoff):")
print(generated_text_lm1)

generated token:  ['<s>', '<s>', '<s>', 'of']
generated token:  ['<s>', '<s>', '<s>', 'of', 'his']
generated token:  ['<s>', '<s>', '<s>', 'of', 'his', 'learned']
generated token:  ['<s>', '<s>', '<s>', 'of', 'his', 'learned', 'and']
generated token:  ['<s>', '<s>', '<s>', 'of', 'his', 'learned', 'and', 'hast']
generated token:  ['<s>', '<s>', '<s>', 'of', 'his', 'learned', 'and', 'hast', 'given']
generated token:  ['<s>', '<s>', '<s>', 'of', 'his', 'learned', 'and', 'hast', 'given', 'him']
generated token:  ['<s>', '<s>', '<s>', 'of', 'his', 'learned', 'and', 'hast', 'given', 'him', '.']
generated token:  ['<s>', '<s>', '<s>', 'of', 'his', 'learned', 'and', 'hast', 'given', 'him', '.', 'and']
generated token:  ['<s>', '<s>', '<s>', 'of', 'his', 'learned', 'and', 'hast', 'given', 'him', '.', 'and', 'power']
generated token:  ['<s>', '<s>', '<s>', 'of', 'his', 'learned', 'and', 'hast', 'given', 'him', '.', 'and', 'power', 'at']
generated token:  ['<s>', '<s>', '<s>', 'of', 'his', 'learn

In [87]:
# Text Generator for LM2 (Interpolation + Add-k Smoothing)
def generate_text_lm2(start_tokens, lambdas, k, max_length=20):
    """
    Generate text using LM2 (Interpolation + Add-k Smoothing).
    
    Parameters:
    - start_tokens: List of tokens to start the text generation (e.g., ["<s>", "<s>", "<s>"]).
    - lambdas: Interpolation weights.
    - k: Add-k smoothing parameter.
    - max_length: Maximum length of the generated sequence.
    
    Returns:
    - Generated text as a string.
    """
    generated_tokens = start_tokens[:]

    while len(generated_tokens) < max_length:
        context = tuple(generated_tokens[-3:])  # Use the last 3 tokens as context for 4-grams
        next_token_probs = {}

        # Compute probabilities for the next token
        for token in unigram_counts.keys():
            ngram = context + token#(token,)
            prob = interpolated_probability(ngram, lambdas, k)  # Use LM2's interpolated probability
            if prob > 0:
                next_token_probs[token] = prob

        # Normalize probabilities
        total_prob = sum(next_token_probs.values())
        normalized_probs = {token: prob / total_prob for token, prob in next_token_probs.items()}

        # Sample the next token
        next_token = random.choices(list(normalized_probs.keys()), weights=normalized_probs.values())[0]

        # Stop if end token is generated
        if next_token == "</s>":
            break

        # generated_tokens.append(next_token)
        generated_tokens.extend(list(next_token))

    return " ".join(generated_tokens[3:])  # Remove the initial <s> tokens


# Generate text using LM2
best_lambda = (0.4, 0.3, 0.2, 0.1)  # Example best lambdas (from tuning)
best_k = 0.5  # Example best k (from tuning)
generated_text_lm2 = generate_text_lm2(start_tokens, best_lambda, best_k)
print("\nGenerated Text (LM2 - Interpolation + Add-k Smoothing):")
print(generated_text_lm2)



Generated Text (LM2 - Interpolation + Add-k Smoothing):
1:22 weakness hollow 26:22 presence 5:2 difference narrow tempted secrets sepulchre contained keep overthrown ascended 2:14 weighed
